In [1]:
import joblib
filename = "results/history/ResNet50_1_history.sav"
history = joblib.load(filename)
history

{'loss': [0.5024577379226685,
  0.22246168553829193,
  0.2143031507730484,
  0.21363767981529236,
  0.2136019915342331,
  0.2136247307062149,
  0.2136353999376297,
  0.21364416182041168,
  0.2136487513780594,
  0.21364587545394897],
 'auc': [0.48173078894615173,
  0.48525479435920715,
  0.4727247953414917,
  0.4666137099266052,
  0.4795754551887512,
  0.47894996404647827,
  0.4734995663166046,
  0.46566733717918396,
  0.46642807126045227,
  0.4657370150089264],
 'precision': [0.08275000005960464,
  0.08275000005960464,
  0.08275000005960464,
  0.08275000005960464,
  0.08275000005960464,
  0.08275000005960464,
  0.08275000005960464,
  0.08275000005960464,
  0.08275000005960464,
  0.08275000005960464],
 'recall': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 'f1_score': [array([0.3125997 , 0.19148934, 0.05825243, 0.19727892, 0.05555555,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.03302753, 0.        , 0.        , 0.        ],
       

In [2]:
!pwd

/home/jovyan/ChestXray-14


In [3]:
import tensorflow as tf
from modules.dataset import Dataset

In [5]:
train, test = Dataset().get_kfold(1, sample=True)
test

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 15), dtype=tf.int64, name=None))>

In [14]:
y_true=[]
for X,y in test:
    for label in y:
        y_true.append(label)
y_true = tf.Variable(y_true)
y_true

<tf.Variable 'Variable:0' shape=(200, 15) dtype=int64, numpy=
array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])>

In [19]:
import tensorflow as tf
import wandb
from wandb.keras import WandbCallback

from modules.dataset import Dataset
from modules.models import Model

# Constant variables
NAME = "ResNet50"
EPOCHS = 10
NUM_FOLDS = 5

dataset = Dataset()
fold_num = 1

# Dataset
train_dataset, test_dataset = dataset.get_kfold(fold_num, sample=True)

# Modeling
model = tf.keras.models.load_model("results/models/resnet50_fold1.h5")

In [20]:
y_preds = model.predict(test_dataset)

2022-09-15 09:55:47.300044: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-09-15 09:55:48.192134: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-09-15 09:55:48.193251: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-09-15 09:55:48.193304: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-09-15 09:55:48.194344: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-09-15 09:55:48.194550: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


      5/Unknown - 5s 31ms/step

2022-09-15 09:55:49.484938: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


13/13 [==============================] - 5s 86ms/step


In [30]:
(y_preds > 0.4).astype('int16')

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int16)

In [47]:
y_preds

array([[0.48581904, 0.10406391, 0.03509307, ..., 0.07648436, 0.08108151,
        0.01197154],
       [0.48581904, 0.10406391, 0.03509307, ..., 0.07648436, 0.08108151,
        0.01197154],
       [0.48581904, 0.10406391, 0.03509307, ..., 0.07648436, 0.08108151,
        0.01197154],
       ...,
       [0.48581904, 0.10406391, 0.03509307, ..., 0.07648436, 0.08108151,
        0.01197154],
       [0.48581904, 0.10406391, 0.03509307, ..., 0.07648436, 0.08108151,
        0.01197154],
       [0.48581904, 0.10406391, 0.03509307, ..., 0.07648436, 0.08108151,
        0.01197154]], dtype=float32)

In [31]:
m = tf.keras.metrics.AUC(multi_label=True)
m.update_state(y_true, y_preds)
m.result().numpy()

0.5

In [ ]:
import inspect
inspect.getmembers(m)

In [141]:
""" Interesting Keys
>>* m._thresholds
>>> m._non_trainable_weights
>>> m._tracking_metadata
>>> m._unconditional_checkpoint_dependencies
>>* m._unconditional_dependency_names

>>* m.true_positives
>>* m.true_negatives
>>* m.false_positives
>>* m.false_negatives

>>* m.num_thresholds
>>* m.variables

for name,value in m._unconditional_checkpoint_dependencies:
    print(value.numpy())
"""

m.variables == list(m.weights)

True

In [165]:
for value, value_2 in zip(m.variables, [m.true_positives.numpy().tolist(), 
                                        m.true_negatives.numpy().tolist(),
                                        m.false_positives.numpy().tolist(),
                                        m.false_negatives.numpy().tolist()]):
    assert value.numpy().tolist() == value_2, "Not Equal"

15